In [19]:
from bs4 import BeautifulSoup
import requests

In [ ]:
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'}

categorys = [329251,329226,329240,329279,329294,329323]
# 근력운동 : 329251
# 요가/필라테스 : 329226
# 요가복 : 329240
# 유산소운동 : 329279
# 스트레칭/균형 : 329294
# 헬스소품/보호대 : 329323

lis = []
for category in categorys:
    for i in range(1,22):
        url = f'https://www.coupang.com/np/categories/{category}?page={i}'
        res = requests.get(url)
        text = res.text
        soup = BeautifulSoup(text)
        product_check = soup.find('p', {'class':'no-list-item'})
        if product_check == None:
            # ul  = soup.find("ul", {"id":"productList"})
            lis.append(soup.findAll("li", {"class":"baby-product renew-badge"}))
        else:
            break

In [43]:
# 상품이름 , 종류 , 출처 , 링크 , 할인 , 정상가 , 할인가 , 리뷰개수 , 평점
for page_list in lis:

    for list in page_list:
        #링크
        list_url = list.find('a', {'class':'baby-product-link'})['href']
        url = f'coupang.com/{list_url}'
        # print(url)

        #상품 이름
        product_name = list.find('div', {'class':'name'}).text.strip().replace(' ','')
        print(product_name)

        # 출처
        source = '쿠팡'

        # 가격과 할인율
        base_price = list.find('del', {'class':'base-price'})
        discount_price = list.find('strong', {'class':'price-value'}).text
        discount_rate = list.find('span', {'class':'discount-percentage'})
        if discount_rate != None:
            discount_rate = discount_rate.text.replace('%','')
            base_price = base_price.text.strip()
            print(base_price)
            print(f'할인율 : {discount_rate}%')
        else:
            print(discount_price)
            print('할인율 : 0%')

        #리뷰개수
        bracket = str.maketrans('()','  ')
        review_count = list.find('span', {'class':'rating-total-count'})
        if review_count == None :
            print('리뷰: 0')

        else:
            review_count = review_count.text.translate(bracket).strip()
            print(f'리뷰: {review_count}')
        #평점
        rate = list.find('em', {'class':'rating'})
        if rate == None:
            print('평점 : 0')
        else:
            rate = rate.text
            print(f'평점 : {rate}')
        print('-------------------------------------------')



아이워너흠집방지안전철봉
23,000
할인율 : 32%
리뷰: 8916
평점 : 4.5
-------------------------------------------
하디로어안전한2중잠금장치프리미엄가정용문틀철봉,블랙
62,000
할인율 : 43%
리뷰: 2827
평점 : 4.5
-------------------------------------------
코멧스포츠파워푸쉬업바2p
13,900
할인율 : 47%
리뷰: 2589
평점 : 5.0
-------------------------------------------
코멧스포츠육각덤벨아령블랙,5kg,2개입
14,900
할인율 : 22%
리뷰: 631
평점 : 4.5
-------------------------------------------
코멧스포츠풀업밴드중,퍼플,중(2단계)
13,900
할인율 : 31%
리뷰: 4934
평점 : 5.0
-------------------------------------------
코멧스포츠풀업밴드강,그린,강(3단계)
17,900
할인율 : 28%
리뷰: 3624
평점 : 5.0
-------------------------------------------
세라밴드탄성저항밴드,레드
9,900
할인율 : 10%
리뷰: 10200
평점 : 4.5
-------------------------------------------
세라밴드탄성저항밴드,BLUE
12,950
할인율 : 18%
리뷰: 10200
평점 : 4.5
-------------------------------------------
코멧문틀흠집방지안전철봉1개,실버
21,900
할인율 : 29%
리뷰: 1202
평점 : 4.5
-------------------------------------------
이고웰프리미엄무게조절덤벨바벨세트,20Kg
39,000
할인율 : 25%
리뷰: 60
평점 : 4.5
-------------------------------------------
코멧클래식논슬립아령3kg,2개,그레이
1

'coupang.com//vp/products/306146810?itemId=965291522&vendorItemId=5369727509&sourceType=CATEGORY&categoryId=329125'